# Лабораторная работа 4. Линейная регрессия
Вариант 32

Выполнил Шульга Артём группа P33111

In [10]:
import pandas as pd

data = pd.read_csv("california_housing_train.csv")
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


### Статистика по датасету
Выведем имя колонки, количество элементов, среднее значение, среднее отклонение, минимальное, максимальное значение и квантили 0,25, 0,75, 0,9 соответственно

In [11]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


### Предобработка данных
Так как у нас отсутствуют NaN и иные виды пустых значений (согласно описанию датасета), а также отсутствуют категориальные признаки, то мы просто нормализуем наши данные

In [9]:
data = (data - data.min()) / (data.max() - data.min())

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,1.000000,0.175345,0.274510,0.147885,0.198945,0.028364,0.077454,0.068530,0.107012
1,0.984064,0.197662,0.352941,0.201608,0.294848,0.031559,0.075974,0.091040,0.134228
2,0.975100,0.122210,0.313725,0.018927,0.026847,0.009249,0.019076,0.079378,0.145775
3,0.974104,0.116897,0.254902,0.039515,0.052142,0.014350,0.037000,0.185639,0.120414
4,0.974104,0.109458,0.372549,0.038276,0.050435,0.017405,0.042921,0.098281,0.104125
...,...,...,...,...,...,...,...,...,...
16995,0.008964,0.854410,1.000000,0.058389,0.060987,0.025337,0.060516,0.128081,0.198764
16996,0.007968,0.866100,0.686275,0.061869,0.081782,0.033381,0.076303,0.139170,0.131960
16997,0.004980,0.988310,0.313725,0.070515,0.082247,0.034782,0.074823,0.174577,0.182682
16998,0.004980,0.984060,0.352941,0.070384,0.085506,0.036296,0.078441,0.102054,0.145981


In [14]:
import random
RANDOM_STATE = 13
TRAIN_SIZE = 0.8
MIN_COEFF = -20
MAX_COEFF = 20
MAX_ITERATIONS = 100

def create_coeffs() -> []:
    coeffs = []

    for a in range (len(data.columns)+1):
        coeffs.append(0.0)
    
    return coeffs

best_coeffs = []
best_score = 1

def fit(X, y):
    stop_counter = 0
    while True:
        coeffs = create_coeffs()
        for i in range(coeffs):
            coeffs[i] = random.uniform(MIN_COEFF, MAX_COEFF)
        
        result = pd.DataFrame
        for i in range(len(coeffs)-1):
            result += coeffs[i] * X.iloc[:, i]
        result += coeffs[len(coeffs)-1]

        rss = ((y - result)**2).sum()

        if (rss < best_score):
            best_score = rss
            best_coeffs = coeffs
        else:
            stop_counter+=1

        if (stop_counter >= MAX_ITERATIONS):
            return
def predict(X, y):
    result = pd.DataFrame
    for i in range(len(best_coeffs)-1):
        result += best_coeffs[i] * X.iloc[:, i]
    result += best_coeffs[len(best_coeffs)-1]

    

    

random.seed(RANDOM_STATE)

data = data.sample(frac=1)
train, labels = data.drop('median_house_value', axis=1), data['median_house_value']
X_train, X_test = train[:int(TRAIN_SIZE * train.shape[0])], train[int(TRAIN_SIZE * train.shape[0]):]
y_train, y_test = labels[:int(TRAIN_SIZE * labels.shape[0])], labels[int(TRAIN_SIZE * labels.shape[0]):]

fit(X_train, y_train)

#print(model.score(X_test, y_test))

In [16]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
5275,-118.14,33.96,38.0,590.0,139.0,620.0,132.0,2.1731
817,-117.07,33.00,4.0,6242.0,1258.0,2211.0,1116.0,4.2500
4757,-118.08,33.82,30.0,2636.0,652.0,3412.0,649.0,2.8095
14918,-122.23,37.81,52.0,1433.0,229.0,612.0,213.0,4.7708
15602,-122.35,37.94,34.0,1880.0,459.0,1358.0,422.0,1.6571
...,...,...,...,...,...,...,...,...
4741,-118.08,33.89,35.0,1071.0,290.0,1412.0,274.0,3.1917
7145,-118.32,33.83,51.0,2399.0,516.0,1160.0,514.0,3.8456
7870,-118.40,34.16,45.0,1176.0,250.0,471.0,228.0,2.3333
14335,-122.10,37.41,33.0,6277.0,1274.0,3025.0,1211.0,5.4721
